<h1>Character Level GPT on Text Data</h1>

### Edit the filename below to train the GPT model on the corpus. Select "Run" -> "Run All".

In [ ]:
%%scorep_env
SCOREP_ENABLE_TRACING=1
SCOREP_ENABLE_PROFILING=0
SCOREP_TOTAL_MEMORY=3g

In [ ]:
#%env SCOREP_KERNEL_PERSISTENCE_MODE MEMORY

In [ ]:
%env

In [ ]:
%%switch_serializer
cloudpickle

In [ ]:
%%scorep_python_binding_arguments
--noinstrumenter

In [ ]:
filename = "fairy_test.txt"

In [ ]:
%%execute_with_scorep
import scorep
import logging

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%d/%m/%Y %H:%M:%S",
    level=logging.INFO)

from utils import set_seed
set_seed(42)

import numpy as numpy
import torch
import torch.nn as nn
from torch.nn import functional as F

import math
from torch.utils.data import Dataset

class CharDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print("data has %d characters, %d unique." % (data_size, vocab_size))

        self.stoi = {ch:i for i, ch in enumerate(chars)}
        self.itos = {i:ch for i, ch in enumerate(chars)}
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx : idx+self.block_size+1]
        dix = [self.stoi[s] for s in chunk]

        x = torch.tensor(dix[:-1], dtype = torch.long)
        y = torch.tensor(dix[1:], dtype = torch.long)
        return x, y

with scorep.instrumenter.enable():
    block_size = 32

    text = open("./{}".format(filename), "r").read()
    train_dataset = CharDataset(text, block_size)

    from model import GPT, GPTconfig
    mconf = GPTconfig(train_dataset.vocab_size, train_dataset.block_size,
                      n_layer=1, n_head=8, n_embd=512)
    model = GPT(mconf)

    from trainer import Trainer, TrainerConfig

    tconf = TrainerConfig(max_epochs=1, batch_size=512, learning_rate=6e-4,
                          lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                          num_workers=4)
    trainer = Trainer(model, train_dataset, None, tconf)
    trainer.train()

    torch.save(model.state_dict(), "./saved_models/trained_gpt_model")